# Case Studies: Creating LAMMPS Simulations of Hydrocarbon polymers

In this example, we use NNMDKit to create LAMMPS data and input files for molecular dynamics (MD) simulations of hydrocarbon polymers. We focus only on hydrocarbon polymers at this stage because our neural network force field was trained against mostly hydrocarbon polymeric systems  (thanks to **Christopher Kuenneth** for developing the force field!)


## Step 1: Importing NNMDKit and defining SMILES strings

In [1]:
import nnmdkit

smiles = [
    '*CC*', '*CC(*)C', '*CC(*)CC', '*CC(*)CCC', '*CC(*)CCCC', '*CC(*)c1ccccc1'
]

First, NNMDKit should be able to imported if it's installed correctly. Secondly, we put SMILES strings of polymers of interest into a list, which we can later loop over to create input files and folders for each polymeric system.

## Step 2: Creating `System` objects and corresponding LAMMPS data files

In [2]:
for s in smiles:
    sys = nnmdkit.System(smiles=s, mw=1000, ntotal=3000, density=0.85)
    data = sys.write_data(output_dir=s)


For each system/SMILES string, we initialize a `System` object, and for each `System`, four attributes have to be defined, including:
- `smiles` - polymer SMILES string (use * as repeat unit connecting points)
- `mw` - Polymer molecular weight
- `ntotal` - Total number of atoms
- `density` - Initial density of the system ($g/cm^3$)

Here, SMILES strings are provided through looping over the list defined in the first step; molecular weight and total number of atoms are chosen as 1000 and 3000, respectively; initial density is set to $0.85g/cm^3$ which is the density of polyethlyene observed from previous MD simulations.

After `System`'s are defined, we call the `write_data` function with the output directory (`output_dir`) specified to create the LAMMPS data file for each system. Here, we name each folder with the system's SMILES string. We will later put all necessary files for a LAMMPS run to the same folder. Note that the `write_data` function returns the data file name, which is fetched by `data` in this example. This `data` string will later be used to initialize `LAMMPS` objects.

NNMDKit currently uses EMC package to generate initial polymer amorphous structures. Representative snapshots of inital structures are shown below

<p align="center">
    <img src='./img/Example_snapshots.png' width="60%"/>
</p>

## Step 3: Creating `Lammps` objects and corresponding LAMMPS input files

In [ ]:
Tmax = 800
Pmax = 100
Tfinal = 300
Pfinal = 1

eq_step = [
    ['nvt', 50000, Tmax],
    ['npt', 50000, Tfinal, Pmax],
    ['npt', 50000, Tfinal, Pfinal],
    ['nvt', 50000, Tmax],
    ['npt', 50000, Tfinal, Pmax],
    ['npt', 50000, Tfinal, Pfinal],
    ['nvt', 50000, Tmax],
    ['npt', 50000, Tfinal, Pmax],
    ['npt', 50000, Tfinal, Pfinal]
]

We first define a list named `eq_step` that specify equilibration procedure which will be discussed below.
In this example, a 9-step equilibration procedure is used:

1. 50 ps NVT simulation at 800 K
2. 50 ps NPT simulation at 400 K and 100 bar
3. 50 ps NPT simulation at 400 K and 1 bar
4. 50 ps NVT simulation at 800 K
5. 50 ps NPT simulation at 400 K and 100 bar
6. 50 ps NPT simulation at 400 K and 1 bar
7. 50 ps NVT simulation at 800 K
8. 50 ps NPT simulation at 400 K and 100 bar
9. 50 ps NPT simulation at 400 K and 1 bar

In [3]:
for s in smiles:
    lmp = nnmdkit.Lammps(data_fname=data, NN_POTENTIAL='potential_saved')
    lmp.add_procedure('equilibration', eq_step=eq_step)
    lmp.add_procedure('Tg_measurement', Tinit=400, Tfinal=100, Tinterval=20, step=100000)
    lmp.write_input(output_dir=s)

Then, for each system, we initialize a `Lammps` object, and the following attributes are defined:
- `data_fname` - name of the LAMMPS data file
- `NN_POTENTIAL` - name of the neural network parameter file
- `atom_style` (*optional*) - argument of [LAMMPS atom_style](https://docs.lammps.org/atom_style.html); default: 'full'
- `units` (*optional*) - argument of [LAMMPS units](https://docs.lammps.org/units.html); default: 'metal'
- `timestep` (*optional*) - argument of [LAMMPS timestep](https://docs.lammps.org/timestep.html); default: 0.001
- `neighbor_skin` (*optional*) - argument of [LAMMPS neighbor](https://docs.lammps.org/neighbor.html); default: 2.0
- `neighbor_every` (*optional*) - argument of [LAMMPS neighbor_modify](https://docs.lammps.org/neigh_modify.html); default: 1
- `thermo` (*optional*) - argument of [LAMMPS thermo](https://docs.lammps.org/thermo.html); default: 100,
- `pair_style` (*optional*) - argument of [LAMMPS pair_style](https://docs.lammps.org/pair_style.html); default: `'nn'`
- `element` (*optional*) - a string of the list of elements in the system; default: `'C H'`

Once the `Lammps` object is defined. We can add simulation procedures by using the `add_procedure` function. Here, we add two procedures: `'equilibration'` and `'Tg_measurement'`. 

For `'equilibration'`, we specify equilibration steps by defining a list `eq_step` on top. Each equilibration step is defined as a list within the `eq_step` list. Several things have to be specified in order in each step/list: 

1. `nvt` or `npt` ensemble
2. Duration of the step
3. Temperature
4. Pressure if `npt` is selected

For `'Tg_measurement'`, we specify several properties:

- `Tinit` - Initial temperature of the cooling process
- `Tfinal` - Final temperature of the cooling process
- `Tinterval` - Temperature interval
- `step` - Simulation duration of each temperature step

In this example, we started the simulations from 400 K and performed 0.1 ns NPT MD simulations at every 20 K step until 100 K. System density is calculated at every temperature step and a text file `temp_vs_density` will be produced with all density recorded at each temperature step.

## Step 4: Creating `Job` objects and corresponding pbs files

In [4]:
for s in smiles:
    job = nnmdkit.Job(jobname=s,
                      project='GT-rramprasad3-CODA20',
                      nodes=3,
                      ppn=24,
                      walltime='96:00:00',
                      LAMMPS_EXEC='~/p-rramprasad3-0/NNLMP/lmp')
    job.write_pbs(output_dir=s)

For each system, we initialize a `Job` object, and the following attributes are defined:
 - `jobname` - Job name
 - `project` - Project name to which resource is charged
 - `nodes` - Number of nodes
 - `ppn` - Number of processors
 - `walltime` - Maximum run time
 - `LAMMPS_EXEC` - Directory of the LAMMPS executable file (with `pair_style nn` registered)

After job objects are defined, we can generate pbs files with output directory specified. With these pbs files, we submit jobs on supercomputers.

# Outcome
This example script prepares you all LAMMPS data and input files needed to run MD simulations and measure glass transition temperature of six basic hydrocarbon polymers. Files for each system are put into folders named by their polymer SMILES strings as shown below:

<p align="center">
    <img src='./img/Tutorial_outcome.png' width="15%"/>
</p>